# Classificação de avaliação IMDB

## Autor

Lucas Gabriel Bezerra - 180125770

GitHub: [lucasgbezerra](https://github.com/lucasgbezerra/)

## Objetivo

O objetivo desse artigo é treinar um modelo capaz de identificar os textos de reviews de filmes do IMDB e classificá-las de acordo com o sentimento como positiva ou negativa.

## Motivo

A fim de aplicar conhecimentos de NLP uma das formas de utilizá-lo é a partir da classificação textual. Além disso, o uso de reviews em sites prestadores de serviços tem se tornado cada dia mais comum (Play Store, IMDB, Reclame Aqui), classificar o sentimento de reviews e encaminhá-lo para setores específicos da empresa para tratar da questão envolvida na review pode acelerar o processo de comunicação entre empresa e cliente.

## Instalação de depêndencias

In [1]:
! pip install transformers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.0 MB/s eta 0:00:00a 0:00:01


Para realizar o deploy, irei utilizar o *notebook_login* do huggingface para acessar minha conta e poder salvar o modelo.

Dessa forma, será feita a instalação do pacote *huggingface_hub* e a utilização do login dele.

Ao executar a função *notebook_login()* um prompt aparece com um campo para adicionar um token de acesso disponibilizado pelo huggingface.

In [2]:
!pip install huggingface_hub

In [3]:
from huggingface_hub import notebook_login

notebook_login()

## Preparando Dataset

A base de dados utilizada esta disponível no [kaggle](https://www.kaggle.com/datasets/columbine/imdb-dataset-sentiment-analysis-in-csv-format).

Possui 3 CSVs, irei utilizar apenas 2 pra simplificar a criação dos datasets, o resultado obtido se mostrou satisfatorio dessa maneira.


In [4]:
import pandas as pd

df_train = pd.read_csv('/kaggle/input/imdb-dataset-sentiment-analysis-in-csv-format/Train.csv')
df_valid = pd.read_csv('/kaggle/input/imdb-dataset-sentiment-analysis-in-csv-format/Valid.csv')

O conjunto de dados tem 2 campos: 

- `text`: o texto de review do filme.
- `label`: um valor de 0 para avaliações negativas ou 1 para avaliações positivas

In [5]:
df_train

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1
...,...,...
39995,"""Western Union"" is something of a forgotten cl...",1
39996,This movie is an incredible piece of work. It ...,1
39997,My wife and I watched this movie because we pl...,0
39998,"When I first watched Flatliners, I was amazed....",1


In [6]:
df_train.describe(include='object')

,text
count,40000
unique,39723
top,"Hilarious, clean, light-hearted, and quote-wor..."
freq,4


Transformando o dataframe em dataset utilizando a lib *datasets*

In [7]:
from datasets import Dataset,DatasetDict

ds_train = Dataset.from_pandas(df_train)
ds_train

Dataset({
    features: ['text', 'label'],
    num_rows: 40000
})

## Pré-processamento do texto (Tokenizando e Numeralização)

O processo de tokenização é fundamental no NLP. Ele se resume em converter o texto em lista de palavras (podendo ser em caracteres,ou substrings, dependendo da granularidade do modelo utilizado).

Após realizar a tokenização, será obtido um vocabulário de palavras únicas presentes no dataset. Como um modelo de deep learning espera números como inputs essas palavras(tokens) são convertidas em números.

O modelo utilizado nesse artigo será o *distilbert-base-uncased*.

O DistilBERT base model é uma versão do oriunda do BERT base model. Ele não diferencia caixa alta e caixa baixa, é mais rápido e menor que o BERT.

In [8]:
from transformers import AutoTokenizer

model_dbu = "distilbert-base-uncased"

toks = AutoTokenizer.from_pretrained(model_dbu)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Função para tokenizar o texto e truncar sequências, se necessário, para que não sejam maiores que o input aceito pelo modelo DistilBERT.

In [9]:
def tokenizer_function(examples):
    return toks(examples["text"], truncation=True)

Aplicando a função a todos os elementos do dataset com a função map.

In [10]:
tokenized_imdb = ds_train.map(tokenizer_function, batched=True)

  0%|          | 0/40 [00:00<?, ?ba/s]

### Exemplo de tokenização e Numeralização

Segue um exemplo de como o *tokenize()* separa o texto em "tokens" e como cada um recebe um número no vocabulário.

In [11]:
toks.tokenize('This movie is an incredible piece of work')

['this', 'movie', 'is', 'an', 'incredible', 'piece', 'of', 'work']

In [12]:
toks.vocab['movie']

3185

Agora observe um dos textos "tokenizados" do dataset utilizado

In [13]:
row = tokenized_imdb[0]
row['text'], row['input_ids']

('I grew up (b. 1965) watching and loving the Thunderbirds. All my mates at school watched. We played "Thunderbirds" before school, during lunch and after school. We all wanted to be Virgil or Scott. No one wanted to be Alan. Counting down from 5 became an art form. I took my children to see the movie hoping they would get a glimpse of what I loved as a child. How bitterly disappointing. The only high point was the snappy theme tune. Not that it could compare with the original score of the Thunderbirds. Thankfully early Saturday mornings one television channel still plays reruns of the series Gerry Anderson and his wife created. Jonatha Frakes should hand in his directors chair, his version was completely hopeless. A waste of film. Utter rubbish. A CGI remake may be acceptable but replacing marionettes with Homo sapiens subsp. sapiens was a huge error of judgment.',
 [101,
  1045,
  3473,
  2039,
  1006,
  1038,
  1012,
  3551,
  1007,
  3666,
  1998,
  8295,
  1996,
  8505,
  12887,
 

Agora será criado um lote de exemplos usando o *DataCollatorWithPadding()*, esse lote será utilizado posteriormente como um dos parametrôs do treinamento do modelo.

In [14]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=toks)

### Separando o dataset em teste e treino e validação

Para realização do treinamento e testar o modelo após o dataset será dividido em 2 partes, uma contendo 20% dos dados (dados de teste) e a outra com os 80% restantes.

Pra isso foi utilizado o método *train_test_split()*.

In [15]:
dds = tokenized_imdb.train_test_split(0.20, seed=42)
dds

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 32000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 8000
    })
})

Realizando a limpeza das labels do dataframe de validação, e convertendo em dataset para utilização posterior ao treinamento do modelo.

Como pode ser visto o dataset de validação é criado a partir de outro dataframe com dados diferentes do dataframe que gerou o dataset de treino e teste

In [16]:
df_valid.drop(['label'], axis=1, inplace=True)
ds_valid = Dataset.from_pandas(df_valid)

Realizar a separação dos datasets tem um motivo importantissimo.

Isso evita que ocorra o **overfitting** e o **underfitting**

Uma breve explicação:

Um modelo é basicamente um sistema para mapear entradas para saídas. Em suma, um modelo aprende relacionamento entre as entradas (features) e as saídas (labels) a partir de um dataset de treinamento. Depois de treinado o modelo deve ser capaz de receber apenas as entradas (dataset de testes) e ele faz previsões sobre as saídas a partir do que aprendeu no treinamento.

O Modelo mais simples é uma regressão linear e para aumentar a complexidade desse modelo basta aumentar o grau polinomial.

$$
   y = \beta_0x + \beta_1 x² = + \beta_2 x² + ... + \beta_n x^n + \epsilon
$$

![](https://miro.medium.com/max/720/1*pjIp920-MZdS_3fLVhf-Dw.webp)

O grau do polinomial é a representação da flexibilida do modelo. E a partir disso podemos entender o que é o undefit e o overfit de um modelo.

Um modelo underfit será menos flexível, o que o impede de contabilizar os dados como visto na imagem a seguir.


|||
|--|--|
|![](https://miro.medium.com/max/640/1*kZfqaD6hl9iYGYXkMwV-JA.webp)| ![](https://miro.medium.com/max/640/1*2RXJ2O-_c2ukaq5p-WQ9tQ.webp)

A função do modelo na cor laranja, na imagem à esquerda, está acima da função real e do conjunto de dados de treinamento. À direita está a previsão dada pelo modelo. Basicamente o modelo ignora o conjunto de treinamento, pois o modelo underfit tem uma **baixa varância**(depende pouco dos dados de treinamento) e **alto viés** (faz uma forte suposição sobre o comportamento dos dados). Quando o modelo tenta fazer previsões o alto viés o faz ter previsões com alta imprecissão.

|||
|--|--|
|![](https://miro.medium.com/max/640/1*Di7rY6ALXtkhlmlcKRSCoA.webp)| ![](https://miro.medium.com/max/640/1*QzA45ATjeEbwv5f1G99GnQ.webp)

Quando o grau do modelo é aumentado muito, o modelo consegue alcançar todas alterações dos dados de treinamento, essa alta flexibilidade  faz com que o modelo tenha uma ótima precisão nos dados de treinamento, mas os dados possuem ruídos e essa **alta flexibilidade** faz com que o modelo se ajuste a eles. Assim o modelo irá possuir uma **alta variância**, ele basicamente **memoriza os dados de treinamento** e não "aprende" como é o desejado.

## Avaliação

Para avaliar o desempenho do modelo é preciso ter uma métrica. 

Para simplificar a obtenção de um método que possa fazer essa avaliação do modelo, utilizaremos a biblioteca evaluate que possui duzias de métodos de avaliação pra diferentes domínios além do NLP.

A métrica que decide por utilizar foi a métrica de *accuracy**

In [17]:
import evaluate

accuracy = evaluate.load("accuracy")

Essa função utiliza da previsão feita e da label para calcular a métrica utilizada (acurácia).

In [18]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

## Treinamento

Map para converter as labes em palavras significantes e o contrário

In [19]:
id_label = {0: "NEGATIVE", 1: "POSITIVE"}
label_id = {"NEGATIVE": 0, "POSITIVE": 1}

Carregando o modelo pré treinado DistilBERT com o número de labels e o mapeamento delas

In [21]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    model_dbu, 
    num_labels=2, 
    id2label=id_label, 
    label2id=label_id
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

Ao executar o *TrainingArguments* uma key para obter automaticamente os pesos e vies era pedido. Não consegui obter a Key então para desabilitei o registro automático de pesos e vieses.

In [22]:
import os

os.environ["WANDB_DISABLED"] = "true"

Instalando as depenências de git para ser possível fazer o push do arquivo do modelo para o HugginFace

In [24]:
!sudo apt-get install git-lfs
!git lfs install

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 93 not upgraded.
Need to get 3316 kB of archives.
After this operation, 11.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 git-lfs amd64 2.9.2-1 [3316 kB]
Fetched 3316 kB in 0s (6795 kB/s)
Selecting previously unselected package git-lfs.
(Reading database ... 108827 files and directories currently installed.)
Preparing to unpack .../git-lfs_2.9.2-1_amd64.deb ...
Unpacking git-lfs (2.9.2-1) ...
Setting up git-lfs (2.9.

Definindo os **argumentos do treinamento com *TrainingArguments()***
- *output_dir:* Onde salvar o modelo;
- *learning_rate:* taxa de aprendizagem inicial;
- *per_device_train_batch_size:* o tamanho da amostra de treino por GPU/TPU;
- *per_device_eval_batch_size:* o tamanho da amostra de avaliação por GPU/TPU;
- *num_train_epochs:* Número de épocas para o treinamento;
- *weight_decay:* A taxa de decaimento do peso a ser aplicada as camadas;
- *evaluation_strategy:* A estrátegia de avaliação adotada no treinamento;
- *save_strategy:* A estratégia do ponto de salvamento a ser adotada durante o treinamento;
- *load_best_model_at_end:* Carregar ou não o melhor modelo encontrado durante o treinamento (Quando verdadeiro exige que os parâmetros "evaluation_strategy" e "save_strategy" sejam iguais);
- *push_to_hub:* "Push" do modelo para o Hub, quando ele for salvo (depende do "output_dir").

Conhecendo os parâmetros utilizados para a classe *transofrmers.Trainer*:
- *model:* O modelo a ser treinado, avaliado ou utilizado para previsão;
- *args:* Os argumentos para ajustar o treinamento;
- *train_dataset:* O conjunto de dados que será utilizado para o treinamento,
- *eval_dataset:* O conjunto de dados utilizado para avaliação;
- *tokenizer:* O tokenizador usado para pré-processar os dados;
- *data_collator:* A função utilizada para formar um lote a partir de uma lista de elementos de train_dataset ou eval_dataset;
- *compute_metrics:* a função que será utilizada paar computar as métricas na avaliaçã.

In [25]:
training_args = TrainingArguments(
    output_dir="classification_text_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dds["train"],
    eval_dataset=dds["test"],
    tokenizer=toks,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Cloning https://huggingface.co/lucasgbezerra/classification_text_model into local empty directory.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Download file pytorch_model.bin:   0%|          | 7.44k/255M [00:00<?, ?B/s]

...

Clean file runs/Dec14_01-07-56_f68c23e4e125/1670980121.9676027/events.out.tfevents.1670980121.f68c23e4e125.23.…

Clean file training_args.bin:  31%|###       | 1.00k/3.23k [00:00<?, ?B/s]

Clean file runs/Dec14_01-10-26_f68c23e4e125/1670980235.297694/events.out.tfevents.1670980235.f68c23e4e125.23.3…

Clean file pytorch_model.bin:   0%|          | 1.00k/255M [00:00<?, ?B/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [26]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 32000
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 2000
/opt/conda/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will inst

Epoch,Training Loss,Validation Loss,Accuracy
1,0.204700,0.177547,0.932125
2,0.129100,0.201243,0.934125


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 8000
  Batch size = 32
Saving model checkpoint to classification_text_model/checkpoint-1000
Configuration saved in classification_text_model/checkpoint-1000/config.json
Model weights saved in classification_text_model/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in classification_text_model/checkpoint-1000/tokenizer_config.json
Special tokens file saved in classification_text_model/checkpoint-1000/special_tokens_map.json
tokenizer config file saved in classification_text_model/tokenizer_config.json
Special tokens file saved in classification_text_model/special_tokens_map.json


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

/opt/conda/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 8000
  Batch size = 32
Saving model checkpoint to classification_text_model/checkpoint-2000
Configuration saved in classification_text_model/checkpoint-2000/config.json
Model weights saved in classification_text_model/checkpoint-2000/pytorch_model.bin
tokenizer config file saved in classification_text_model/checkpoint-2000/tokenizer_config.json
Special tokens file saved in classification_text_model/checkpo

...

tokenizer config file saved in classification_text_model/tokenizer_config.json
Special tokens file saved in classification_text_model/special_tokens_map.json


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 



Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from classification_text_model/checkpoint-1000 (score: 0.1775466948747635).


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


TrainOutput(global_step=2000, training_loss=0.1904059410095215, metrics={'train_runtime': 1919.2026, 'train_samples_per_second': 33.347, 'train_steps_per_second': 1.042, 'total_flos': 8474585270592768.0, 'train_loss': 0.1904059410095215, 'epoch': 2.0})

O tempo de treinamento foi longo, apesar de utilizar apenas 2 épocas e realizar o treinamento em uma maquina do kaggle com o uso da GPU T4 X2, o treinamento do modelo demorou cerca de 30 minutos. 

Assim como configurado nos argumentos o modelo foi salvo em um [repositório](https://huggingface.co/lucasgbezerra/classification_text_model) com o nome especificado na conta logada no ínicio do artigo.

## Testando o modelo

Vamos verificar o que o modelo prevê a partir do dataset de testes e validação

In [30]:
preds = trainer.predict(dds['test']).predictions.astype(float)
preds

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 8000
  Batch size = 32
/opt/conda/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


array([[ 2.23709369, -2.29509497],
       [-1.78306019,  1.47043931],
       [-1.34510422,  0.93321747],
       ...,
       [-2.40362573,  1.9160639 ],
       [ 0.50562054, -0.47438526],
       [-2.07757139,  1.67809868]])

A previsão do dataset de testes retornou números menores que -1 e maiores que 1. Para melhorar o entendimento sobre os resultados e definir limites vamos utilizar o método clip do numpy para deixar estes números dentre o range 0 à 1, que nos permite visualizar como porcentagem o grau de positivadade ou negatividade das frases avaliadas.

In [31]:
preds = np.clip(preds, 0, 1)
preds

array([[1.        , 0.        ],
       [0.        , 1.        ],
       [0.        , 0.93321747],
       ...,
       [0.        , 1.        ],
       [0.50562054, 0.        ],
       [0.        , 1.        ]])

Agora utilizando outro conjunto de dados, os dados de validaçaão

Primeiro eles serão tokenizados para então poder ocorrer a previsão

In [32]:
toks_valid = ds_valid.map(tokenizer_function, batched=True)

  0%|          | 0/5 [00:00<?, ?ba/s]

In [37]:
preds2 = trainer.predict(toks_valid).predictions.astype(float)
preds2

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 5000
  Batch size = 32


array([[ 1.95127702, -1.99396503],
       [ 2.37554598, -2.5228641 ],
       [ 2.17845559, -2.19129372],
       ...,
       [-1.58796299,  1.19799149],
       [-2.55518889,  2.01890063],
       [-2.28080058,  1.7151109 ]])

Novamente os resultados serão normalizados entre 0 e 1

In [38]:
preds2 = np.clip(preds2, 0, 1)
preds2

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]])

## Resultado do deploy

Para realizar o deploy utilizei o modelo salvo no [HuggingFace](https://huggingface.co/lucasgbezerra/classification_text_model).

E a partir dele gerei um Space com o seguinte app.py:


```python
import gradio as gr

description = "Classificador de sentimento (Positivo, Negativo) de textos de avaliações de filmes"
examples = [
#             lista de exemplos disponíveis para usar o modelo  
        ]

gr.Interface.load("models/lucasgbezerra/classification_text_model",  description=description, examples=examples).launch()
```

Para acessar o deploy basta [clicar aqui](https://huggingface.co/spaces/lucasgbezerra/classification_text)

[![](https://i.imgur.com/GorVTk3.png)](https://huggingface.co/spaces/lucasgbezerra/classification_text)

## Conclusão

Aplicar um modelo pré treinado para NLP é um desafio maior do que o encontrado nos tópicos anteriores de Vision Computer.

A utilização das libs do HugginFace facilitaram bastante, principalmente a nível de entendimento.

A maior dificuldade, quando comparado a Vision Computer está na preparação dos dados. Dados visuais são muito mais simples de serem separados, e erros mais simples de serem notados no conjunto de dados, quando os dados passam a ser textuais a interpretação deles é mais complexa.

Outro ponto a se destacar é o tempo gasto no treinamento, no modelo NLP o tempo gasto em 2 épocas é muito alto, provavelmente muito disso seja devido a quantidade de dados utilizados, cerca de 40 mil textos.

Por último, vale destacar que quanto mais se aprofunda nos parâmetros para a construção do modelo, maior o grau de conhecimento se torna necessário. Aspectos como o underfitting e o overfitting se tornam básicos para entender como seu modelo se porta e como melhorá-lo. A métrica de avaliação se torna mais complexa, e pode ser de diversos tipos, cada uma com um foco e se encaixando melhor em determinado modelo.